In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from warnings import filterwarnings
filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
drive.mount('/content/drive',force_remount=True)
train_data = '/content/drive/My Drive/AI_ML/train.csv'
Orig_df = pd.read_csv(train_data,header="infer",sep=",")
test_data = '/content/drive/My Drive/AI_ML/test.csv'
AV_test_df = pd.read_csv(test_data,header="infer",sep=",")

Mounted at /content/drive


In [3]:
Orig_df.drop(columns=['ID'],inplace=True)
result = AV_test_df['ID']
AV_test_df.drop(columns=['ID'],inplace=True)

In [4]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

Orig_df_filled = pd.DataFrame(imputer.fit_transform(Orig_df),columns=Orig_df.columns)

AV_test_df_filled = pd.DataFrame(imputer.fit_transform(AV_test_df),columns=AV_test_df.columns)

In [7]:
X = Orig_df_filled[['Estimated_Insects_Count', 'Crop_Type', 'Soil_Type',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit', 'Season']]
y = Orig_df_filled['Crop_Damage']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.2)

In [9]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_train_scale = std_scaler.fit_transform(X_train)
X_test_scale = std_scaler.fit_transform(X_test)

In [10]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier()

params = {'max_depth':[3,4,5,6], 'learning_rate':np.arange(0.0,1.0,0.02), 'n_estimators':[100,200,300,400],'min_samples_split':[2,4,6,8],
    'min_samples_leaf':[1,5,8,10,12],'max_leaf_nodes':[5,10,12,15,20],'max_features':np.arange(1,9)}

from sklearn.model_selection import RandomizedSearchCV

rscv = RandomizedSearchCV(xgb_clf,param_distributions=params,scoring='accuracy',cv=5)

rscv.fit(X_train_scale,y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, sc...
       0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
       0.88, 0.9 , 0.92, 0.94, 0.96, 0.98]),
                                        'max_depth': [3, 4,

In [11]:
rscv.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.18, max_delta_step=0, max_depth=5, max_features=1,
              max_leaf_nodes=12, min_child_weight=1, min_samples_leaf=8,
              min_samples_split=6, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [12]:
y_pred_xgb = rscv.best_estimator_.predict(X_test_scale)
#y_pred_train = rscv.best_estimator_.predict(X_train_scale)

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
testing_score = round(accuracy_score(y_test,y_pred_xgb),3)
training_score = round(np.mean((cross_val_score(rscv.best_estimator_, X_train_scale, y_train, cv=5))),3)

In [14]:
print(training_score,testing_score) 

0.847 0.849


In [15]:
AV_scale = std_scaler.fit_transform(AV_test_df_filled)

In [16]:
y_pred_res = rscv.best_estimator_.predict(AV_scale)

In [19]:
AV_test_df = pd.read_csv(test_data,header="infer",sep=",")
result=AV_test_df[['ID']]
result=result[['ID']]
result['Crop_Damage'] = y_pred_res
result = result[['ID','Crop_Damage']]

In [22]:
result.to_csv('/content/drive/My Drive/AI_ML/AV_Agri_XGB1.csv',index=False)

*Deleting the 3 columns: Croptype,soiltype,Season*

In [25]:
Orig_df_filled_FE = Orig_df_filled.drop(columns=['Crop_Type','Soil_Type','Season'])
AV_test_df_filled_FE = AV_test_df_filled.drop(columns=['Crop_Type','Soil_Type','Season'])

In [26]:
X1 = Orig_df_filled_FE[['Estimated_Insects_Count',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit']]
y1 = Orig_df_filled_FE['Crop_Damage']

In [27]:
from sklearn.model_selection import train_test_split
X_train1,X_test1,y_train1,y_test1 = train_test_split(X1,y1,random_state=0,test_size=0.2)

In [28]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_train1_scale = std_scaler.fit_transform(X_train1)
X_test1_scale = std_scaler.fit_transform(X_test1)

In [29]:
xgb_clf1 = XGBClassifier()

params = {'max_depth':[3,4,5,6], 'learning_rate':np.arange(0.0,1.0,0.02), 'n_estimators':[100,200,300,400],'min_samples_split':[2,4,6,8],
    'min_samples_leaf':[1,5,8,10,12],'max_leaf_nodes':[5,10,12,15,20],'max_features':np.arange(1,9)}

from sklearn.model_selection import RandomizedSearchCV

rscv1 = RandomizedSearchCV(xgb_clf1,param_distributions=params,scoring='accuracy',cv=5)

rscv1.fit(X_train1_scale,y_train1)

rscv1.best_estimator_

rscv1.best_params_

y_pred_gb1 = rscv1.best_estimator_.predict(X_test1_scale)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
testing_score1 = round(accuracy_score(y_test1,y_pred_gb1),3)
training_score1 = round(np.mean((cross_val_score(rscv1.best_estimator_, X_train1_scale, y_train1, cv=5))),3)

print(training_score1,testing_score1)



0.846 0.847


FileNotFoundError: ignored

In [30]:
AV_scale_FE = std_scaler.fit_transform(AV_test_df_filled_FE)

y_pred_res1 = rscv1.best_estimator_.predict(AV_scale_FE)


#result1.to_csv('AV_Agri_GB_FE.csv',index=False)

In [31]:
AV_test_df = pd.read_csv(test_data,header="infer",sep=",")
result1=AV_test_df[['ID']]
result1['Crop_Damage'] = y_pred_res1
result1 = result1[['ID','Crop_Damage']]


In [32]:
result.to_csv('/content/drive/My Drive/AI_ML/AV_Agri_XGB_FE.csv',index=False)

In [33]:
AV_test_df_filled_FE['Pesticide_Use_Category'].replace(to_replace=1, value=2, inplace=True)
Orig_df_filled_FE['Pesticide_Use_Category'].replace(to_replace=1, value=2, inplace=True)

In [34]:
X2 = Orig_df_filled_FE[['Estimated_Insects_Count',
       'Pesticide_Use_Category', 'Number_Doses_Week', 'Number_Weeks_Used',
       'Number_Weeks_Quit']]
y2 = Orig_df_filled_FE['Crop_Damage']

from sklearn.model_selection import train_test_split
X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y2,random_state=0,test_size=0.2)

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_train2_scale = std_scaler.fit_transform(X_train2)
X_test2_scale = std_scaler.fit_transform(X_test2)

xgb_clf2 = XGBClassifier()

params = {'max_depth':[3,4,5,6], 'learning_rate':np.arange(0.0,1.0,0.02), 'n_estimators':[100,200,300,400],'min_samples_split':[2,4,6,8],
    'min_samples_leaf':[1,5,8,10,12],'max_leaf_nodes':[5,10,12,15,20],'max_features':np.arange(1,9)}

from sklearn.model_selection import RandomizedSearchCV

rscv2 = RandomizedSearchCV(xgb_clf2,param_distributions=params,scoring='accuracy',cv=5)

rscv2.fit(X_train2_scale,y_train2)

rscv2.best_estimator_

rscv2.best_params_

y_pred_gb2 = rscv2.best_estimator_.predict(X_test2_scale)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
testing_score1 = round(accuracy_score(y_test2,y_pred_gb2),3)
training_score1 = round(np.mean((cross_val_score(rscv2.best_estimator_, X_train2_scale, y_train2, cv=5))),3)



NameError: ignored

In [35]:
print(training_score1,testing_score1)

AV_scale_FE2 = std_scaler.fit_transform(AV_test_df_filled_FE)

y_pred_res2 = rscv2.best_estimator_.predict(AV_scale_FE2)

AV_test_df = pd.read_csv(test_data,header="infer",sep=",")
result2=AV_test_df[['ID']]
result2['Crop_Damage'] = y_pred_res2
result2 = result2[['ID','Crop_Damage']]

0.845 0.847


In [36]:
result.to_csv('/content/drive/My Drive/AI_ML/AV_Agri_XGB_FE2.csv',index=False)

In [37]:
import sklearn.neural_network
from sklearn.model_selection import GridSearchCV
parameter_space = {
    'hidden_layer_sizes': [(100, ),(150, )],
    'activation': ['sigmoid','relu'],
    'solver': ['adam'],
    'alpha': [0.0001,0.001],
    'learning_rate': ['constant','adaptive']
}

mlp_gs = sklearn.neural_network.MLPClassifier(max_iter=100)
rscv_ml = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
rscv_ml.fit(X_train_scale,y_train)

y_pred_mlp = rscv_ml.predict(X_test_scale)
y_train_pred = rscv_ml.predict(X_train_scale)

accuracy_test = sklearn.metrics.accuracy_score(y_test, y_pred_mlp)
accuracy_train = sklearn.metrics.accuracy_score(y_train, y_train_pred)

print(accuracy_test,accuracy_train)
y_pred_Av = rscv_ml.predict(AV_scale)

AV_test_df = pd.read_csv(test_data,header="infer",sep=",")
result2=AV_test_df[['ID']]
result2['Crop_Damage'] = y_pred_Av
result2 = result2[['ID','Crop_Damage']]
result.to_csv('/content/drive/My Drive/AI_ML/AV_Agri_MLP.csv',index=False)

0.8454872833670943 0.8473117069465155


NameError: ignored